In [27]:
"""
This example showcases how simple it is to build image classification networks.
It follows description from this TensorFlow tutorial:
    https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#deep-mnist-for-experts
"""
from sklearn import metrics
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import skflow

In [28]:
### Download and load MNIST data.
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [29]:
### Linear classifier
classifier = skflow.TensorFlowLinearClassifier(
    n_classes=10, batch_size=100, steps=1000, learning_rate=0.01)
classifier.fit(mnist.train.images, mnist.train.labels)

Step #1, avg. loss: 2.64234
Step #101, avg. loss: 1.90145
Step #201, avg. loss: 1.36286
Step #301, avg. loss: 1.10363
Step #401, avg. loss: 0.95203
Step #501, avg. loss: 0.85592
Step #601, epoch #1, avg. loss: 0.78724
Step #701, epoch #1, avg. loss: 0.73279
Step #801, epoch #1, avg. loss: 0.70121
Step #901, epoch #1, avg. loss: 0.65916


TensorFlowLinearClassifier(batch_size=100, continue_training=False,
              early_stopping_rounds=None,
              keep_checkpoint_every_n_hours=10000, learning_rate=0.01,
              max_to_keep=5, n_classes=10, optimizer='SGD', steps=1000,
              tf_master='', tf_random_seed=42, verbose=1)

In [30]:
score = metrics.accuracy_score(mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))
print "Accuracy: %f" % score

Accuracy: 0.860700
Accuracy: 0.860700


In [31]:
### Convolutional network
def max_pool_2x2(tensor_in):
    return tf.nn.max_pool(tensor_in, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
        padding='SAME')

def conv_model(X, y):
    # reshape X to 4d tensor with 2nd and 3rd dimensions being image width and height
    # final dimension being the number of color channels
    X = tf.reshape(X, [-1, 28, 28, 1])
    # first conv layer will compute 32 features for each 5x5 patch
    with tf.variable_scope('conv_layer1'):
        h_conv1 = skflow.ops.conv2d(X, n_filters=32, filter_shape=[5, 5], 
                                    bias=True, activation=tf.nn.relu)
        h_pool1 = max_pool_2x2(h_conv1)
    # second conv layer will compute 64 features for each 5x5 patch
    with tf.variable_scope('conv_layer2'):
        h_conv2 = skflow.ops.conv2d(h_pool1, n_filters=64, filter_shape=[5, 5], 
                                    bias=True, activation=tf.nn.relu)
        h_pool2 = max_pool_2x2(h_conv2)
        # reshape tensor into a batch of vectors
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    # densely connected layer with 1024 neurons
    h_fc1 = skflow.ops.dnn(h_pool2_flat, [1024], activation=tf.nn.relu, keep_prob=0.5)
    return skflow.models.logistic_regression(h_fc1, y)

In [32]:
# Training and predicting
classifier = skflow.TensorFlowEstimator(
    model_fn=conv_model, n_classes=10, batch_size=100, steps=1000,
    learning_rate=0.001)
classifier.fit(mnist.train.images, mnist.train.labels)

Step #1, avg. loss: 3.32872
Step #101, avg. loss: 2.47812
Step #201, avg. loss: 2.21500
Step #301, avg. loss: 2.04412
Step #401, avg. loss: 1.89221
Step #501, avg. loss: 1.75343
Step #601, epoch #1, avg. loss: 1.64950
Step #701, epoch #1, avg. loss: 1.59568
Step #801, epoch #1, avg. loss: 1.67436
Step #901, epoch #1, avg. loss: 1.90646


TensorFlowEstimator(batch_size=100, continue_training=False,
          early_stopping_rounds=None, keep_checkpoint_every_n_hours=10000,
          learning_rate=0.001, max_to_keep=5,
          model_fn=<function conv_model at 0x1021b1b90>, n_classes=10,
          num_cores=4, optimizer='SGD', steps=1000, tf_master='',
          tf_random_seed=42, verbose=1)

In [11]:
score = metrics.accuracy_score(mnist.test.labels, classifier.predict(mnist.test.images))
print('Accuracy: {0:f}'.format(score))

Accuracy: 0.631300
